In [20]:
base_path = "/Users/lorenapacheco/Concordia/Masters/BugReportsMining/"
#base_path = r"C:\Users\loren\Concordia\Masters\BugReportsMining\/"
ochiai_scores_paths_list = [base_path + "ochiaiScores2.0/Ochiai1/", base_path + "ochiaiScores2.0/Ochiai2/", base_path + "ochiaiScores2.0/Ochiai3/", base_path + "ochiaiScores2.0/Ochiai4/"]
#ochiai_scores_paths_list = [base_path + r"ochiaiScores2.0\Ochiai1/", base_path + r"ochiaiScores2.0\Ochiai2/", base_path + r"ochiaiScores2.0\Ochiai3/", base_path + r"ochiaiScores2.0\Ochiai4/"]
out = base_path + "BugReportsMining/Rankings/"
#out = base_path + r"Rankings\/"
data_file_path = base_path + "data/merged_data_production_bug_reports.json"
#data_file_path = base_path + r"data\merged_data_production_bug_reports.json"
bug_metrics_file = base_path + "data/ochiai_metrics_per_bug_defects4j.csv"
#bug_metrics_file = base_path + r"data\ochiai_metrics_per_bug_defects4j.csv"
project_metrics_file = base_path + "data/ochiai_metrics_per_project_defects4j.csv"
#project_metrics_file = base_path + r"data\ochiai_metrics_per_project_defects4j.csv"
fake_failing_tests_file_path = {
    "Ochiai1": base_path + "ochiaiScores2.0/Ochiai1/fake_failing_tests_info.json",
    "Ochiai2": base_path + "ochiaiScores2.0/Ochiai2/fake_failing_tests_info.json",
    "Ochiai3": base_path + "ochiaiScores2.0/Ochiai3/fake_failing_tests_info.json",
    "Ochiai4": base_path + "ochiaiScores2.0/Ochiai4/fake_failing_tests_info.json"
}

In [34]:
import glob
import os
import json
import pandas as pd

def json_file_to_dict(file):
    data = {}
    with open(os.path.join(file), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data

def dict_to_json_file(file, dic, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(folder, file+'.json'), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()

def sort_dict (dictionary):
    return dict(sorted(dictionary.items(), key=lambda item: item[1], reverse = True))

def get_number_of_buggy_methods_in_top_n (ranking_data, n, buggy_methods_list):
    buggy_methods_in_top_n = 0
    for method in buggy_methods_list:
        try:
            method_in_pattern = find_method_in_ranking_data(method, ranking_data)
            if ranking_data[method_in_pattern] <= n:
                buggy_methods_in_top_n +=1
        except:
            continue
    return buggy_methods_in_top_n

def find_method_in_ranking_data(method, ranking_data):
    for m in ranking_data.keys():
        if method.endswith(m):
            return m
    return None

def get_first_buggy_method_in_stack_trace(buggy_methods_list, stack_trace_methods):
    best_position = float('inf')
    for buggy_method in buggy_methods_list:
        for index, st_method in enumerate(stack_trace_methods):
            st_method_id = st_method
            last_dot_index = st_method.rfind('.')
            if last_dot_index != -1:
                st_method_id = st_method[:last_dot_index] + '#' + st_method[last_dot_index + 1:]
            if buggy_method.endswith(st_method_id):
                if (index + 1) < best_position:
                    best_position = index + 1
                break
    if best_position == float('inf'):
        best_position = "not found"
    return best_position


def get_best_classified_buggy_method (ranking_data, buggy_methods_list):
    best_position = float('inf')
    for method in buggy_methods_list:
        try:
            if ranking_data[method] < best_position:
                best_position = ranking_data[method]
        except:
            continue
    if best_position == float('inf'):
        best_position = "not found"
    return best_position

def get_precision_top_n (ranking_data, n, buggy_methods_list):
    buggy_methods_in_top_n = get_number_of_buggy_methods_in_top_n(ranking_data, n, buggy_methods_list)
    precision =  buggy_methods_in_top_n / n
    return precision

def extract_buggy_methods_list (ranking_data, buggyMethods):
    buggy_methods_list = []
    temp=[]
    for file in buggyMethods.keys():
        class_id = file.replace(".java", "")
        class_id = class_id.replace("/", ".")
        class_id = class_id.replace("\/", ".")
        for method_name in buggyMethods[file]:
            method_id = class_id + "#" + method_name
            temp.append(method_id)
    for temp_method_id in temp:
        found = False
        for method_name in ranking_data.keys():
            if temp_method_id.endswith(method_name):
                buggy_methods_list.append(method_name)
                found = True
                break
        if not found:
            buggy_methods_list.append(temp_method_id)
    return buggy_methods_list


def get_average_precision_top_n(n, project, buggyMethods):
    sum = 0
    for bug_id in buggyMethods[project].keys():
        ranking_file = out + ochiai_identificator + "/" + project + "/" + bug_id + ".json"
        ranking_info = json_file_to_dict(ranking_file)
        sum += get_precision_top_n (ranking_info, n, project, bug_id)
    return sum/len(buggyMethods[project].keys())


def get_recall_top_n (ranking_data, n, buggy_methods_list):
    buggy_methods_in_top_n = get_number_of_buggy_methods_in_top_n(ranking_data, n, buggy_methods_list)
    recall = buggy_methods_in_top_n / len(buggy_methods_list)
    return recall

def get_average_recall_top_n(n, project):
    sum = 0
    for bug_id in buggyMethods[project].keys():
        ranking_file = out + ochiai_identificator + "/" + project + "/" + bug_id + ".json"
        ranking_info = json_file_to_dict(ranking_file)
        sum += get_recall_top_n (ranking_info, n, project, bug_id)
    return sum/len(buggyMethods[project].keys())

def get_f1_top_n (ranking_data, n,buggyMethods):
    precision = get_precision_top_n(ranking_data, n, buggyMethods)
    recall = get_recall_top_n(ranking_data, n, buggyMethods)
    try:
        f1 = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        return 0.0
    return f1

def get_average_f1_top_n(n, project,buggyMethods):
    sum = 0
    for bug_id in buggyMethods[project].keys():
        ranking_file = out + ochiai_identificator + "/" + project + "/" + bug_id + ".json"
        ranking_info = json_file_to_dict(ranking_file)
        sum += get_f1_top_n (ranking_info, n, project, bug_id)
    return sum/len(buggyMethods[project].keys())

def get_method_rank(ranking_info, method):
    for method_rank in ranking_info.keys():
        if method.endswith(method_rank):
            return ranking_info[method_rank]
    return float('inf')


def get_mrr (project, ochiai_identificator, project_bugs_data):
    sum_for_mrr = 0
    number_of_bugs = len(project_bugs_data)
    for bug_id in project_bugs_data.keys():
        buggyMethods = project_bugs_data[bug_id]["buggyMethods"]
        buggy_methods_list = extract_buggy_methods_list (ranking, buggyMethods)
        if ochiai_identificator == "stackTraces":
            ranking_info = get_st_raking_dict(project_bugs_data[bug_id]["stack_trace_methods"])
        else:
            ranking_file = out + ochiai_identificator + "/" + project + "/" + bug_id + ".json"
            try:
                ranking_info = json_file_to_dict(ranking_file)
            except FileNotFoundError:
                continue
        if len(ranking_info) == 0: # No ranking info
            if ochiai_identificator != "stackTraces": # Due to the ausence of gzoltar files
                number_of_bugs -= 1
            continue
        best_rank_found = len(ranking_info.keys())
        for buggy_method in  buggy_methods_list:
            if get_method_rank(ranking_info, buggy_method) < best_rank_found:
                best_rank_found = get_method_rank(ranking_info, buggy_method)
        sum_for_mrr += 1/best_rank_found
    mrr = sum_for_mrr/number_of_bugs
    return mrr

def get_st_raking_dict(stack_trace_methods):
    st_method_formated = []
    for st_method in stack_trace_methods:
        last_dot_index = st_method.rfind('.')
        if last_dot_index != -1:
            st_method_id = st_method[:last_dot_index] + '#' + st_method[last_dot_index + 1:]
            st_method_formated.append(st_method_id)
    return {item: index+1 for index, item in enumerate(st_method_formated)}

def create_or_update_bug_metrics_file(bug_metrics):
    # Column names
    columns = ['Project', 'Bug Id', 'Stack Trace (ST) size - N', 'Number of buggy methods',
               'Position of the first buggy method into the ST', 'Precision ST Top N','Recall ST Top N', 'F1 ST Top N',
               'Number of fake failing tests Ochiai1', 'Number of fake passing tests Ochiai1',
               'Position of the first buggy method into the Ochiai1', 'Precision Ochiai1 Top N','Recall Ochiai1 Top N', 'F1 Ochiai1 Top N',
               'Number of fake failing tests Ochiai2', 'Number of fake passing tests Ochiai2',
               'Position of the first buggy method into the Ochiai2', 'Precision Ochiai2 Top N','Recall Ochiai2 Top N', 'F1 Ochiai2 Top N',
               'Number of fake failing tests Ochiai3', 'Number of fake passing tests Ochiai3',
               'Position of the first buggy method into the Ochiai3', 'Precision Ochiai3 Top N','Recall Ochiai3 Top N', 'F1 Ochiai3 Top N',
               'Number of fake failing tests Ochiai4', 'Number of fake passing tests Ochiai4',
               'Position of the first buggy method into the Ochiai4', 'Precision Ochiai4 Top N','Recall Ochiai4 Top N', 'F1 Ochiai4 Top N']

    # Create the dataframe if the file doesn't exist
    if not os.path.isfile(bug_metrics_file):
        df = pd.DataFrame(columns=columns)
        df.to_csv(bug_metrics_file, index=False)
    else:
        # Otherwise, read existing file into dataframe
        df = pd.read_csv(bug_metrics_file)

    # Iterate over the bugs object
    for project in bug_metrics.keys():
        for bug_id in  bug_metrics[project].keys():
            data = {'Project': project, 'Bug Id': bug_id}
            for metric in bug_metrics[project][bug_id].keys():
                data[metric] = bug_metrics[project][bug_id][metric]

            # Create a temporary dataframe for new data
            new_data = pd.DataFrame(data, index=[0])

            # Check if the bug_id and project exist in the dataframe
            mask = (df['Project'] == project) & (df['Bug Id'] == bug_id)

            if not df[mask].empty:
                # If the bug_id and project exist, update the row
                df.loc[mask, columns[2:]] = new_data[columns[2:]].values
            else:
                # If it doesn't exist, append the new data
                df = pd.concat([df, new_data], ignore_index=True)

    # Write the dataframe back to csv
    df.to_csv(bug_metrics_file, index=False)

def create_or_update_project_metrics_file(project_metrics):
    # Column names
    columns = ['Project', 'Map Stack Traces', 'MRR Stack Traces',
               'Map Ochiai1', 'MRR Ochiai1',
               'Map Ochiai2', 'MRR Ochiai2',
               'Map Ochiai3', 'MRR Ochiai3',
               'Map Ochiai4', 'MRR Ochiai4']

    # Create the dataframe if the file doesn't exist
    if not os.path.isfile(project_metrics_file):
        df = pd.DataFrame(columns=columns)
        df.to_csv(project_metrics_file, index=False)
    else:
        # Otherwise, read existing file into dataframe
        df = pd.read_csv(project_metrics_file)

    # Iterate over the bugs object
    for project in project_metrics.keys():
        data = {'Project': project}
        for metric in project_metrics[project].keys():
            data[metric] = project_metrics[project][metric]

        # Create a temporary dataframe for new data
        new_data = pd.DataFrame(data, index=[0])

        # Check if the bug_id and project exist in the dataframe
        mask = (df['Project'] == project)

        if not df[mask].empty:
            # If the bug_id and project exist, update the row
            df.loc[mask, columns[2:]] = new_data[columns[2:]].values
        else:
            # If it doesn't exist, append the new data
            df = pd.concat([df, new_data], ignore_index=True)

    # Write the dataframe back to csv
    df.to_csv(project_metrics_file, index=False)


def get_map(project, ochiai_identificator, project_bugs_data):
    sum_for_map = 0
    number_of_bugs = 0
    for bug_id in project_bugs_data.keys():
        buggyMethods = project_bugs_data[bug_id]["buggyMethods"]
        buggy_methods_list = extract_buggy_methods_list (ranking, buggyMethods)
        if ochiai_identificator == "stackTraces":
            ranking_info = get_st_raking_dict(project_bugs_data[bug_id]["stack_trace_methods"])
        else:
            ranking_file = out + ochiai_identificator + "/" + project + "/" + bug_id + ".json"
            try:
                ranking_info = json_file_to_dict(ranking_file)
            except FileNotFoundError:
                continue
        if len(ranking_info) == 0:
            if ochiai_identificator != "stackTraces":
                continue
        number_of_bugs += 1
        relevant_docs = 0
        sum_for_ap = 0
        buggy_methods_found = 0
        for rank, method in enumerate(sorted(ranking_info, key=ranking_info.get), start=1):
            for buggy_method in buggy_methods_list:
                if buggy_method.endswith(method):
                    relevant_docs += 1
                    precision_at_rank = relevant_docs / rank
                    sum_for_ap += precision_at_rank
                    buggy_methods_found += 1
                    break
        sum_for_map += sum_for_ap / buggy_methods_found if buggy_methods_found else 0
    return sum_for_map / number_of_bugs if number_of_bugs else 0


bugs_data = json_file_to_dict(data_file_path)
bug_metrics = {}
project_metrics = {}
for ochiai_output_folder in ochiai_scores_paths_list:
    folders = ochiai_output_folder.split(os.sep)
    folders = [i for i in folders if i]
    ochiai_identificator= folders[-1].replace("/", "").replace("\/", "")
    fake_failing_tests_info = json_file_to_dict(fake_failing_tests_file_path[ochiai_identificator])
    for bug_report_analysis_file in glob.glob(ochiai_output_folder + "*" + os.sep + "*.json"):

        project = bug_report_analysis_file.split(os.sep)[-2]
        bug_id = bug_report_analysis_file.split(os.sep)[-1].replace(".json","")

        print("---------------")
        print(ochiai_identificator + " - " + project + " - " + bug_id)
        if "buggyMethods" not in bugs_data[project][bug_id].keys() or bugs_data[project][bug_id]["buggyMethods"] == {}:
            print("No buggy methods. Skipping it")
            continue
        buggyMethods = bugs_data[project][bug_id]["buggyMethods"]
        stack_trace_files = bugs_data[project][bug_id]["stack_trace_files"]
        stack_trace_methods = bugs_data[project][bug_id]["stack_trace_methods"]
        ochiai_scores_data = json_file_to_dict(bug_report_analysis_file)
        ochiai_scores_data = sort_dict(ochiai_scores_data)
        if len(ochiai_scores_data) == 0:
            print("No Ochiai scores for this bug due to gzoltar files problem. Skipping it")
            continue

        ranking = {}
        actual_pos = 1
        repetition_count = 0
        previous_score = -1
        for method in ochiai_scores_data.keys():
            if ochiai_scores_data[method] != previous_score:
                actual_pos = actual_pos + repetition_count
                repetition_count = 0
                previous_score = ochiai_scores_data[method]
                ranking [method] = actual_pos
                actual_pos += 1
            else:
                repetition_count +=1
                ranking [method] = actual_pos

        dict_to_json_file(bug_id, ranking, out + ochiai_identificator + "/" + project + "/")

        buggy_methods_list = extract_buggy_methods_list (ranking, buggyMethods)
        st_ranking = get_st_raking_dict(stack_trace_methods)
        N = len(stack_trace_methods)
        if project not in bug_metrics.keys():
            bug_metrics[project] = {}
        if bug_id not in bug_metrics[project].keys():
            bug_metrics[project][bug_id] = {}
        bug_obj = bug_metrics[project][bug_id]
        if project not in fake_failing_tests_info.keys() or bug_id not in fake_failing_tests_info[project].keys():
            continue
        if 'Stack Trace (ST) size - N' not in bug_obj.keys():
            bug_obj['Stack Trace (ST) size - N'] = N
            bug_obj['Number of buggy methods'] = len(buggyMethods)
            bug_obj['Position of the first buggy method into the ST'] = get_first_buggy_method_in_stack_trace(buggy_methods_list, stack_trace_methods)
            bug_obj['Precision ST Top N'] = get_precision_top_n (st_ranking, N, buggy_methods_list )
            print(get_precision_top_n (st_ranking, N, buggy_methods_list ))
            bug_obj['Recall ST Top N'] = get_recall_top_n (st_ranking, N, buggy_methods_list )
            bug_obj['F1 ST Top N'] = get_f1_top_n (st_ranking, N, buggy_methods_list )
        bug_obj['Position of the first buggy method into the ' + ochiai_identificator] = get_best_classified_buggy_method (ranking, buggy_methods_list)
        bug_obj['Precision ' + ochiai_identificator + ' Top N'] = get_precision_top_n (ranking, N, buggy_methods_list )
        bug_obj['Recall ' + ochiai_identificator + ' Top N'] = get_recall_top_n (ranking, N, buggy_methods_list )
        bug_obj['F1 ' + ochiai_identificator + ' Top N'] = get_f1_top_n (ranking, N, buggy_methods_list )
        bug_obj['Number of fake failing tests '+ ochiai_identificator] =  fake_failing_tests_info[project][bug_id]["fake_failing_tests_number"]
        bug_obj['Number of fake passing tests '+ ochiai_identificator] =  fake_failing_tests_info[project][bug_id]["fake_passing_tests_number"]
        print("Number of buggy methods - " + str(bug_obj['Stack Trace (ST) size - N']))
        print("Stack trace size - " + str(N))
        print("Position of the first buggy method in the stack trace - " + str(bug_obj['Position of the first buggy method into the ST']))
        print("Position of the first buggy method into the Ochiai classification - " + str(bug_obj['Position of the first buggy method into the ' + ochiai_identificator]))
        print("Precision Ochiai Top " + str(N) + " - " + str(bug_obj['Precision ' + ochiai_identificator + ' Top N']))
        print("Recall Ochiai Top " + str(N) + " - " + str(bug_obj['Recall ' + ochiai_identificator + ' Top N']))
        print("F1 Ochiai Top " + str(N) + " - " + str(bug_obj['F1 ' + ochiai_identificator + ' Top N']))
        print("Precision Stack Trace Top " + str(N) + " - " + str(bug_obj['Precision ST Top N']))
        print("Recall Stack Trace Top " + str(N) + " - " + str(bug_obj['Recall ST Top N']))
        print("F1 Stack Trace Top " + str(N) + " - " + str(bug_obj['F1 ST Top N']))
        bug_metrics[project][bug_id] = bug_obj

    print("")
    print("")
    print("======= " + ochiai_identificator + " =======")
    for project in bugs_data.keys():
        if project == "Lang" or project == "Math" or  project ==  "fastjson" or project == "junit4":
            continue
        if project not in project_metrics.keys():
            project_metrics[project] = {}
        proj_obj = project_metrics[project]

        if 'Map  Stack Traces' not in proj_obj.keys():
            proj_obj['Map Stack Traces'] = get_map (project, "stackTraces",bugs_data[project])
            proj_obj['MRR Stack Traces'] = get_mrr (project, "stackTraces",bugs_data[project])

        proj_obj['Map ' + ochiai_identificator] = get_map (project, ochiai_identificator, bugs_data[project])
        proj_obj['MRR ' + ochiai_identificator] = get_mrr (project, ochiai_identificator,bugs_data[project])
        project_metrics[project] = proj_obj

        print("")
        print("----")
        print(project)
        print("Map Ochiai - " + str(proj_obj['Map ' + ochiai_identificator]))
        print("Map stack traces - " + str(proj_obj['Map Stack Traces']))
        print("MRR Ochiai- " + str(proj_obj['MRR ' + ochiai_identificator]))
        print("MRR Stack Traces- " + str(proj_obj['MRR Stack Traces']))
    print("==============")
    print("")

create_or_update_bug_metrics_file(bug_metrics)
create_or_update_project_metrics_file(project_metrics)
print("Done")


---------------
Ochiai1 - Closure - 37
0.07692307692307693
Number of buggy methods - 13
Stack trace size - 13
Position of the first buggy method in the stack trace - 2
Position of the first buggy method into the Ochiai classification - 295
Precision Ochiai Top 13 - 0.0
Recall Ochiai Top 13 - 0.0
F1 Ochiai Top 13 - 0.0
Precision Stack Trace Top 13 - 0.07692307692307693
Recall Stack Trace Top 13 - 0.5
F1 Stack Trace Top 13 - 0.13333333333333336
---------------
Ochiai1 - Closure - 125
0.025
Number of buggy methods - 40
Stack trace size - 40
Position of the first buggy method in the stack trace - 9
Position of the first buggy method into the Ochiai classification - 1113
Precision Ochiai Top 40 - 0.0
Recall Ochiai Top 40 - 0.0
F1 Ochiai Top 40 - 0.0
Precision Stack Trace Top 40 - 0.025
Recall Stack Trace Top 40 - 0.5
F1 Stack Trace Top 40 - 0.047619047619047616
---------------
Ochiai1 - Closure - 152
No Ochiai scores for this bug due to gzoltar files problem. Skipping it
---------------
Och